## Catalogue Fitting

Synference has a method specifically for fitting catalogues of sources - `fit_catalogue`.

This is designed to be a flexible and fast way to fit your full catalogue.

It can handle:
1. Transforming observations to match the expected model features.
2. Prior predictive checks/outlier detection to remove sources that are unlikely to be well modelled.
3. Optional imputation of missing data.
4. Rapid posterior inference using trained model.
5. Optional SED recovery (see [this notebook](sed_recovery.ipynb) for more details).
6. Returning structured results.

In [ ]:
?SBI_Fitter.fit_catalogue